<a href="https://colab.research.google.com/github/paulokuriki/siim_2022_turnkey_segment/blob/main/ML_Ed_AutoSeg_001_with_Submitter_05_june_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![picture](https://raw.githubusercontent.com/paulokuriki/siim_2022_turnkey_segment/main/siim22_banner_generic_800.png)

# Segmenting kidneys in MRI images

## SIIM Github Repo

### Task

In this tutorial we will explore implementing an automated segmentation method. The challenge is to create a method for performing segmentation of kidneys in MRI images of mice.  The data set consists of 150 axial MRI images from a variety of mouse models.  The kidneys were manually segmented and you will use these manual segmentations to develop your own deep learning model. You can read more about this data set in [Edwards et al. (2021)](https://pubmed.ncbi.nlm.nih.gov/32828755/).

### Requirements

  1. Basic understanding of machine learning and deep learning

  2. Programming in Python

### Learning Objectives

  1. To train a deep learning segmentation model and learn the basics of keras
  
  2. To learn to interpret the results of training and validation
  
  3. To identify beneficial avenues for improving peformance.

### Acknowledgments

This notebook was developed by the SIIM ML Education Sub-Committee and authored by Timothy L. Kline and Darryl E. Wright. The dataset is made available at: 'https://github.com/ImagingInformatics/machine-learning/tree/master/Education/KerasSegmentation'. Details related to this study are available in: https://kidneyinternational-online.org/action/showPdf?pii=S0085-2538%2820%2930957-1

If you have questions, please contact: kline.timothy@mayo.edu



---


#Setting your Team Name

In [ ]:
# Enter your Team Name below
team = "SIIM 2022"

In [ ]:
print("Your Team Name is:", team)



---


# Required libraries

Note: you can pip install directly in your jupyter notebook. Just add an exclamation point at the beginning (this allows you to execute Terminal commands within a jupyter notebook).

Example: `!pip install keras-unet`


In [ ]:
# Install missing components for colab
!pip install keras_unet

In [ ]:
# Libraries
import os
import numpy as np
import nibabel as nib
import tensorflow as tf
import matplotlib.pyplot as plt
from skimage import measure
from skimage.transform import resize
from keras_unet.metrics import dice_coef
from keras_unet.models import custom_unet
from keras_unet.losses import jaccard_distance
from sklearn.model_selection import train_test_split
import requests

def sub_model(team, hyperparam):
  url='https://bids-class.azurewebsites.net/submit-model'
  hyperparam['team']=team
  hyperparam['ModelKey']='SIIM_AutoSeg_001'
  x=requests.post(url,data=hyperparam)
  if x.status_code==200:
      print(f"Model Submitted Successfully for team {team}")
  else:
      print("Failed to Submit")

### Clone the repository

In [ ]:
!git clone 'https://github.com/TLKline/AutoTKV_MouseMRI/' 


---
# Data preparation and familiarization

## Read in the data set

We are now in a position to read in the data.  We will start by reading in the names of the files and corresponding segmentations.  We will also create an array called ```indices``` that we will use to index into the ```images``` and ```segmentations``` arrays in the next step. 

In [ ]:
data_path = 'AutoTKV_MouseMRI/AllTrainingImages/'
images = []
segmentations = []
for f in os.listdir(data_path):
  if 'label' in f:
    continue
  else:
    images.append(f)
    segmentations.append(f.replace('.nii', '-label.nii'))
    
print("Sample dataset files:")
print("Study:",images[0])
print("Segmentation Mask:", segmentations[0])
images = np.array(images)
segmentations = np.array(segmentations)

indices = np.array(range(len(images))) # we will use this in the next step.

In [ ]:
indices

---
## Train-validation-test split

### Use ```sklearn.model_selection.train_test_split``` to split the data into training and test partitions.

The next step is to define the training, validation and test sets.  Use ```sklearn.model_selection.train_test_split``` and the ```indices``` array to create two new arrays with indices corresponding to a training partition with 75% of the data set and a test partition with 25% of the data.

In [ ]:
# split indices into training and test partitions
train, test = train_test_split(indices, test_size=0.25, random_state=42) 

### Use ```sklearn.model_selection.train_test_split``` to split a validation set from the training partition.
Define a training and validation split. Use ```sklearn.model_selection.train_test_split``` and the ```train``` array to create two new arrays with indices corresponding to a training partition with 75% of the data set and a validation parition with 25% of the data.

In [ ]:
# split train into training and validation partitions
train, valid = train_test_split(train, test_size=0.25, random_state=42) 

Let's see how these ```train```, ```valid``` and ```test``` arrays look.

In [ ]:
print(train)
print(valid)
print(test)

Let's double check that there is no overlap in the partitions.

In [ ]:
assert list(np.intersect1d(train, valid)) == []
assert list(np.intersect1d(train, test)) == []
assert list(np.intersect1d(valid, test)) == []

Below is a helper function that demonstrates how to read a .nii.gz file with [nibabel](https://nipy.org/nibabel/).

In [ ]:
def read_nifti_file(path):
  nib_object = nib.load(path)
  nib_data = nib_object.get_fdata()
  nib_data = np.rot90(nib_data,3)
  return nib_data

We will use this function to help familiarise ourselves with the format of the data.

## Read in an example image and the corresponding segmentation.

Use the above function to read in the first image and segmentation from the training partition and inspect that shape of the returned matrix.

In [ ]:
# Read in the first study image and segmentation from the training partition
example_image = read_nifti_file(data_path+images[train][0]) 
example_segmentation = read_nifti_file(data_path+segmentations[train][0])  

print(example_image.shape, example_segmentation.shape)

## Visualise the first slice from both the image and segmentation

In [ ]:
fig = plt.figure(figsize=(20,10))
ax1 = fig.add_subplot(121)
# Display the first slice from the image
ax1.imshow(example_image[:,:,0], cmap='gray') 
ax1.axis('off')
ax2 = fig.add_subplot(122)
# Display the first slice from the segmentation
ax2.imshow(example_segmentation[:,:,0])  
ax2.axis('off')

## Visualise the tenth slice from both the image and segmentation

In [ ]:
fig = plt.figure(figsize=(20,10))
ax1 = fig.add_subplot(121)
# Display the tenth slice from the image
ax1.imshow(example_image[:,:,10], cmap='gray') 
ax1.axis('off')
ax2 = fig.add_subplot(122)
# Display the tenth slice from the segmentation
ax2.imshow(example_segmentation[:,:,10])  
ax2.axis('off')

---
# Image Preprocessing

Below is a second helper function that relies on ```read_nifti_file``` to read the images and segmentation files to create the data matrices we require when working with neural networks.  This function also implements resizing of the image slices to a default of 64-by-64 pixels.

## Why do we resize the images?

  1. To ensure all image slices are the same size to feed to the U-Net.
  
  2. To reduce the dimensionality of the data for faster training.


In [ ]:
def retrieve_images_and_segmentations(data_path, images, segmentations, size=(128,128)):
  x = []
  y = []
  for i in range(len(images)):
    seg = read_nifti_file(data_path+segmentations[i])
    img = read_nifti_file(data_path+images[i])
    assert img.shape == seg.shape
    seg = resize(seg,(size[0], size[1], seg.shape[-1]))
    img = resize(img,(size[0], size[1], img.shape[-1]))
    for j in range(seg.shape[-1]):
      # ignore slices that don't have a segmentation
      if np.sum(seg[:,:,j]) == 0:
        continue
      x.append(img[:,:,j])
      y.append(seg[:,:,j])
  x = np.array(x)[:,:,:,np.newaxis]
  y = np.clip(np.array(y)[:,:,:,np.newaxis], 0, 1)

  # randomly shuffle slices
  m = x.shape[0]
  order = np.random.permutation(m)

  return x[order], y[order]

The function above also ignores image slices with no corresponding segmentation and treats each slice as an independent training example.  This "bakes in" two assumptions:

  1. that slices with no segmentation provide no useful information for training
  
  2. that the information in one slice has no dependence on the preceeding or subsequent images in the image volume.
  
Think about the implications of these assumptions and why we chose them here.  Which of these assumptions might have the greatest impact on the performance we might be able to achieve?

## Get the data matrices for each of the partitions.

Use ```retrieve_images_and_segmentations``` to retrieve the image and segmentation matrices we need to develop and test our model.  Use the ``train```, ```valid``` and ```test``` arrays we defined above.

In [ ]:
# Get the matrices for the training partition
x_train, y_train = retrieve_images_and_segmentations(data_path, images[train], segmentations[train]) 
print(x_train.shape, y_train.shape)

In [ ]:
# Get the matrices for the validation partition
x_valid, y_valid = retrieve_images_and_segmentations(data_path, images[valid], segmentations[valid]) 
print(x_valid.shape, y_valid.shape)

In [ ]:
# Get the matrices for the test partition
x_test, y_test = retrieve_images_and_segmentations(data_path, images[test], segmentations[test]) 
print(x_test.shape, y_test.shape)

## Visualize the training data

Let's perform a sanity check to ensure that the data matrices we have just created look correct before we try training a model.

It is common to visualise the segmentations as contours overlaid on the image.  To do this we use ```skimage.measure.find_contours``` to find contours in the segmentation masks.  We will also modify this function later to visualize the results of the model we train.

In [ ]:
def visualise_data(x, y):
  n=4
  dim = int(np.ceil(np.sqrt(n)))
  fig = plt.figure(figsize=(10,10))
  for i in range(n):
    ax = fig.add_subplot(dim, dim, i+1)
    ax.imshow(x[50*i,:,:,0], cmap='gray')
    contours = measure.find_contours(y[50*i,:,:,0], .99)
    for idx, contour in enumerate(contours):
      ax.plot(contour[:,1], contour[:,0], color='#FB3640', lw=4, label=f'kidney {idx+1}')
      ax.legend()
    ax.axis('off')

Here we visualise the first 4 images from the training partition.

In [ ]:
visualise_data(x_train, y_train)

---
# Training the model

We are now ready to train a model to segment the kidneys.

To do this we will use a [U-Net](https://arxiv.org/pdf/1505.04597.pdf) architecture.  We are also going to use the [Keras](https://keras.io/) Python deep learning API to build and train a model.  There are great starter guides to Keras [here](https://keras.io/getting_started/), lots of code examples [here](https://keras.io/examples/) and even an example of how to build a U-Net-like architecture from scratch in Keras [here](https://keras.io/examples/vision/oxford_pets_image_segmentation/).

In this notebook, we are going to focus on the very high level API and take advantage of the [karolzak/keras-unet github repository](https://github.com/karolzak/keras-unet) that implements a number of U-Net type architectures which you install in the first TODO for this notebook.

The high level API requires three steps:

  1. instantiate a [Model](https://keras.io/api/models/model/#model-class) object.
  
  2. compile the model
  
  3. fit the model to the data
  
In general, the Model object exposes methods similar to the sklearn API such as ```.fit``` and ```.predict``` along with other neural network specific methods.


### 1. Instantiate a custom_unet object using the correct input_shape

To start we will instantiate a [custom_unet](https://github.com/karolzak/keras-unet/blob/master/keras_unet/models/custom_unet.py) object from the [karolzak/keras-unet repository](https://github.com/karolzak/keras-unet).  Use the correct ```input_shape``` given our data set and leave all other parameters as defaults.

### 2. Compile the model

Use the ```jaccard_distance``` as the loss function, ```dice_coef``` as a metric to track and use the ```tf.keras.optimizers.Adam``` optimizer with a learning rate of 3e-4.

### 3. Fit the model 

Finally, use your ```training set``` to fit your model and use the ```validation_data``` parameters of the fit method to track the performance on the validation set. Train the  model for 10 epochs.  Assign the result of fit to the ```history``` variable.

In [ ]:
# Instantiate the custom_unet object
model = custom_unet(input_shape=(128, 128, 1)) 

# Compile the model as described above
model.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=3e-4), 
  loss=jaccard_distance,
  metrics=[dice_coef])

# Fit the model to the training data, track performance on the validation set and train it for 5 epochs.
history = model.fit(x_train, y_train, 
                    validation_data=(x_valid, y_valid), 
                    epochs=10)

---
# Evaluating model performance

Here we inspect the returned training history.  
The ```history.history``` object tracks the loss and metrics for each epoch.

In [ ]:
print(history.history)

## Learning curves

### Plot both the training and validation loss learning curves

In [ ]:
# Plot the training and validation losses learning curves
plt.plot(range(len(history.history['loss'])), history.history['loss'], '-', label='loss') 
plt.plot(range(len(history.history['val_loss'])), history.history['val_loss'], '-', label='val_loss')
plt.legend(loc="upper right")
plt.xlabel('epoch')
plt.title('Jaccard loss')
plt.show()

### Plot both the training and validation Dice coefficients

In [ ]:
# Plot the training and validation dice_coef learning curves
plt.plot(range(len(history.history['dice_coef'])), history.history['dice_coef'], '-', label='dice_coef') 
plt.plot(range(len(history.history['val_dice_coef'])), history.history['val_dice_coef'], '-', label='val_dice_coef') 
plt.legend(loc="upper left")
plt.xlabel('epoch')
plt.title('Dice coefficient')
plt.show()

### In the cell below briefly describe what you can conclude from these learning curves.  What is the simplest change we could make to the training proceedure to help improve performance?

### Based on the training curves, what changes can you make to the training procedure?

For one, train the model for more epochs

---
## Evaluating segmentation performance

### Use the trained model to predict on the training set.

In [ ]:
# Predict on the training set
y_pred_train = model.predict(x_train) 
y_pred_train = y_pred_train.astype('float') # casting to type float to avoid ValueError later

### Calculate the Dice coefficient between the manual and predicted segmentations for the training set

In [ ]:
print("Dice in train set:", np.array(dice_coef(y_train, y_pred_train.astype('float'))))

### Similarly calculate the Dice coefficient on the validation set

In [ ]:
# Predict on the validation set
y_pred_valid = model.predict(x_valid) 
y_pred_valid = y_pred_valid.astype('float') # casting to type float to avoid ValueError later
print("Dice in validation set:", np.array(dice_coef(y_valid, y_pred_valid.astype('float'))))

What do you make of this comparison of Dice coefficients measured on the training and validation sets? Is this performance good enough? Let's gather more evidence by visualising some of the segmentations.

Below is a modification of the ```visualise_data()``` method we used above. This modified version allows to visualise both the groundtruth (manual) and predicted segmentations.  The groundtruth segmentation is shown in red and the predicted segmentaiton is shown in blue.

In [ ]:
def visualise_data_and_prediction(x, y, y_pred, dataset = ''):
  n=4
  dim = int(np.ceil(np.sqrt(n)))
  fig = plt.figure(figsize=(10,10))
  for i in range(n):
    ax = fig.add_subplot(dim, dim, i+1)
    ax.imshow(x[50*i,:,:,0], cmap='gray')
    contours = measure.find_contours(y[50*i,:,:,0], .99)
    for idx, contour in enumerate(contours):
      if idx == 0:
        ax.plot(contour[:,1], contour[:,0], color='#FB3640', lw=4, label=dataset+' groundtruth')
        ax.legend()
      else:
        ax.plot(contour[:,1], contour[:,0], color='#FB3640', lw=4)
    contours = measure.find_contours(y_pred[50*i,:,:,0], .99)
    for idx, contour in enumerate(contours):
      if idx == 0:
        ax.plot(contour[:,1], contour[:,0], color='#35A7FF', lw=4, label=dataset+' predicted')
        ax.legend()
      else:
        ax.plot(contour[:,1], contour[:,0], color='#35A7FF', lw=4)        
    ax.axis('off')

In [ ]:
visualise_data_and_prediction(x_train, y_train, y_pred_train, 'train set')


In [ ]:
visualise_data_and_prediction(x_valid, y_valid, y_pred_valid, 'validation set')

### Given the evidence we have gathered do you think we are in the high bias or high variance regime? Provide your answer and justification in the cell below. 

# What are you seeing: High bias or high variance? Justify your answer.

**High bias**. Both the training and validation dice coefficients are low and similar.  Visually, the model is performing poorly on both the training and validation sets.

# Improving performance.

### Now it's your time!

Implement changes to the training procedure based on that you analysed earlier then replicate it below.  Ensure the changes you make runs in a reasonable time (< 10 mins). 

As guidance, in Google Colab, training process takes ~20 seconds per epoch.  This may be significantly different for you.

In [ ]:
############################################################################################

# Change the parameters below and run this cell again

hyperparam = {
    'LearningRate': 3e-4,
    'BatchSize': 32,
    'Epochs': 10,
    'ImageSize': 64,
    'BatchNorm': True,
    'Filters': 16,
    'Dropout': 0.1
}


############################################################################################


model = custom_unet(input_shape=(hyperparam['ImageSize'], hyperparam['ImageSize'], 1), 
                    dropout=hyperparam['Dropout'], 
                    use_batch_norm=hyperparam['BatchNorm'],
                    filters=hyperparam['Filters']
                    ) 

model.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=hyperparam['LearningRate']), 
  loss=jaccard_distance,
  metrics=[dice_coef])

if x_train.shape[1] != hyperparam['ImageSize']:
  x_train, y_train = retrieve_images_and_segmentations(data_path, images[train], segmentations[train], size=(hyperparam['ImageSize'], hyperparam['ImageSize'])) 
  x_valid, y_valid = retrieve_images_and_segmentations(data_path, images[valid], segmentations[valid], size=(hyperparam['ImageSize'], hyperparam['ImageSize'])) 
  x_test, y_test = retrieve_images_and_segmentations(data_path, images[test], segmentations[test], size=(hyperparam['ImageSize'], hyperparam['ImageSize'])) 

history = model.fit(x_train, y_train, 
                    validation_data=(x_valid, y_valid), 
                    epochs=hyperparam['Epochs'], batch_size=hyperparam['BatchSize'])

plt.plot(range(len(history.history['loss'])), history.history['loss'], '-', label='loss') 
plt.plot(range(len(history.history['val_loss'])), history.history['val_loss'], '-', label='val_loss') 
plt.legend(loc="upper right")
plt.xlabel('epoch')
plt.title('Jaccard loss')
plt.show()

plt.plot(range(len(history.history['dice_coef'])), history.history['dice_coef'], '-', label='dice_coef')
plt.plot(range(len(history.history['val_dice_coef'])), history.history['val_dice_coef'], '-', label='val_dice_coef') 
plt.legend(loc="upper left")
plt.xlabel('epoch')
plt.title('Dice coefficient')
plt.show()

y_pred_train = model.predict(x_train) 
y_pred_train = y_pred_train.astype('float') 
print("Dice in train set:", np.array(dice_coef(y_train, y_pred_train.astype('float'))))
visualise_data_and_prediction(x_train, y_train, y_pred_train, 'train set')

y_pred_valid = model.predict(x_valid) 
y_pred_valid = y_pred_valid.astype('float') 
print("Dice in validation set:", float(dice_coef(y_valid, y_pred_valid.astype('float'))))
visualise_data_and_prediction(x_valid, y_valid, y_pred_valid, 'validation set')

y_pred_test = model.predict(x_test) 
y_pred_test = y_pred_test.astype('float') 
hyperparam['metric'] = float(dice_coef(y_test, y_pred_test.astype('float')))
print("Dice in test set:", hyperparam['metric'])
visualise_data_and_prediction(x_test, y_test, y_pred_test, 'test set')

sub_model(team, hyperparam)

# Analysing your results

### Did your performance improve? What is the greatest source of error now, bias or variance?

### Analyse your dice coefficient and compare your results.

### Then, go back to the section "Improving Performance" above to tune different hyperparameters and try to improve your metric.

## Below are suggestions for how this experiment could be modified.

  1. explore the keras_unet [github repository](https://github.com/karolzak/keras-unet). What parameters of the model could we change to further improve performance? How would you setup an experiment to find the best settings for these parameters?

  2. replicate the analysis of [Edwards et al. (2021)](https://reader.elsevier.com/reader/sd/pii/S0085253820309571?token=8C46B51948D5C9822571903C855A46B2A114B98D334C1F1FD510E5FB512D933E2D9D6C4DB40EF0B60E29118E61F6F0A6&originRegion=us-east-1&originCreation=20210823192742) comparing the automatic segmentations to the manual segmentations as closely as you can given the information you have access to.  Is there any bias in how the model segments in comparison to manual segmentations?
  